# 2D Beamformer
An exploration of signal processing theory

In [12]:
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

import sys
import os
import numpy as np

cwd = os.path.abspath(".")
if ("sandbox" in cwd):
    os.chdir(os.path.abspath(".."))
    cwd = os.path.abspath(".")

print ("CWD = " + cwd)
sys.path.append(cwd + "/bin")


# matplotlib.use('nbagg')
import matplotlib.pyplot as plt

CWD = /Users/Devin/Documents/Caltech/DSA/DSAbeamformer


### Create array containing positions of antennas

Antennas can be distributed linearly, randomly, or a mix of both

In [4]:
N = 64
max_pos = 250
Antenna_positions = 'Linear'

pos = np.zeros((N,3))

if Antenna_positions == 'Grid':
    x = np.linspace(-max_pos,max_pos,int(np.floor(np.sqrt(N))))
    y = np.linspace(-max_pos,max_pos,int(np.floor(np.sqrt(N))))

    xv, yv = np.meshgrid(x,y)
    pos[:,0] = xv.ravel()
    pos[:,1] = yv.ravel()
elif Antenna_positions == 'Linear':
    pos[:,0] = np.linspace(-max_pos,max_pos, N)
    pos[:,1] = 0
    pos[:,2] = np.random.uniform(size=(N)) - 1
elif Antenna_positions == 'Random':
    pos[:,0] = 2*max_pos*np.random.uniform(size=(N)) - max_pos
    pos[:,1] = 2*max_pos*np.random.uniform(size=(N)) - max_pos
    pos[:,2] = np.random.uniform(size=(N)) - 1
elif Antenna_positions == 'Linear with Random':
    pos = np.linspace(-max_pos,max_pos,N)
    pos[0] = np.random.uniform(-max_pos,max_pos)
    pos[1] = np.random.uniform(-max_pos,max_pos)
    pos[2] = np.random.uniform(-max_pos,max_pos)

fig = plt.figure(figsize= (10,4))

ax = plt.subplot(121)
ax2 = plt.subplot(122)
ax.scatter(pos[:,0], pos[:,1])
ax.set_xlim([-max_pos, max_pos])
ax.set_ylim([-max_pos, max_pos])
ax.set_xlabel("x")
ax.set_ylabel("y")

ax2.scatter(pos[:,0], pos[:,2])
ax2.set_xlabel("x")
ax2.set_ylabel("z")

plt.show()

### Create Fourier coefficient matrix

In [5]:
c = 299792458.0
zero_pt = 0              # Can be used if the first few channels have been flagged
gpu = 0                  # Determines the bandwidth in any gpu
tot_channels = 2048
n_gpus = 8               # Number of GPUs in system
bw_per_channel = (1.53 - 1.28)/tot_channels
N_BEAMS = 256
N_FREQ = tot_channels / n_gpus


field_of_view_half_angle = 1.5

if Antenna_positions != "Linear":
    ang = np.linspace(-field_of_view_half_angle*np.pi/180.0, field_of_view_half_angle*np.pi/180.0, int(np.sqrt(N_BEAMS))) #beam angles

    T, P = np.meshgrid(ang, ang)

    theta = T.ravel()
    phi = P.ravel()
else:
    theta = np.linspace(-field_of_view_half_angle*np.pi/180.0, field_of_view_half_angle*np.pi/180.0, N_BEAMS) #beam angles
    phi = np.zeros(len(theta))

# print theta

freq = [1.53 - (zero_pt + gpu * tot_channels/(n_gpus-1) + i)* bw_per_channel for i in range(N_FREQ)]

form_beams_with_freq = range(256)
A = np.zeros((len(form_beams_with_freq), N_BEAMS, N), dtype=np.complex64) # Allocate space

'''
Calculate the fourier coefficients

Note we use np.round so that each number is ~ an 8-bit integer
'''
for k, f in enumerate(form_beams_with_freq):
    if (f%10 == 0): print(str(f) + ","),
    wavelength = c/(freq[f]*1e9)
    for i in range(N_BEAMS):
        for j in range(N):
            A[k, i,j] = np.round(127.0*np.exp(-2.0j*np.pi*(pos[j,0]*np.sin(theta[i]) + pos[j,1]*np.sin(phi[i])) /wavelength))/127.0

0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,


### Generate the test signals

In [6]:
if Antenna_positions == "Linear":
    eval_pts = 1024 #number of test signals
    angles = np.linspace(-field_of_view_half_angle*np.pi/180.0, field_of_view_half_angle*np.pi/180.0, eval_pts)
    T_angles = angles
    P_angles = np.zeros(eval_pts)
else:
    eval_pts = 3721 #number of test signals
    angles = np.linspace(-field_of_view_half_angle*np.pi/180.0, field_of_view_half_angle*np.pi/180.0, int(np.sqrt(eval_pts)))
    t, p = np.meshgrid(angles,angles)    
    T_angles = t.ravel()
    P_angles = p.ravel()
    
N_AVERAGING = 1
file_name = "2d_pydata_na_%d_eval_%d_N_%d_%s"% (N_AVERAGING, eval_pts, N, Antenna_positions)
print(file_name)

n_angles = len(T_angles)

try:
    out = np.load("bin/" + file_name + ".npy")
    print("Loading from Memory: " + file_name)
except:
    out = np.zeros((N_BEAMS, eval_pts))

    '''
    Note we use np.round so that each number is ~ an 4-bit integer

    This loop can take a while to calculate
    '''
    for k, f in enumerate(form_beams_with_freq):
        print(str(k) + ","),
        wavelength = c/(freq[f]*1e9)
        for jj in range(n_angles):
            signal = [np.round(7*np.exp(2*np.pi*1j*(pos[i,0]*np.sin(T_angles[jj]) + pos[i,1]*np.sin(P_angles[jj]))/wavelength)) for i in range(N)]
            out[:,jj] += 2*N_AVERAGING*np.abs(np.dot(A[k,:,:], signal))**2
    np.save("bin/" + file_name, out)
    print("\nSaving to Memory: " + file_name)

2d_python_data_navg_1_eval_1024_N_64_Linear
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 2

IOError: [Errno 2] No such file or directory: 'bin/2d_python_data_navg_1_eval_1024_N_64_Linear.npy'

#### Plot beams patterns over top of eachother

In [8]:
# mpld3.disable_notebook()
%matplotlib tk

'''
Change "eval_pts" if more resolution is desired
'''

fig = plt.figure(figsize=(10,4))
fig.suptitle(Antenna_positions + " Locations")
ax = plt.subplot(121,projection='polar')
ax2 = plt.subplot(122)

for jj in range(256):
    ax.plot(angles, np.real(out[jj]))
    ax2.plot(angles*180/np.pi, np.real(out[jj]))
    
ax2.axis([-3.6, -3.4, -100, 2.0*max(out[0])])
ax2.set_xlabel("Angle [deg]")
ax2.set_ylabel("Response Amplitude")

plt.show()
# mpld3.enable_notebook()

ValueError: x and y must have same first dimension, but have shapes (61,) and (3721,)

#### Plot Fourier coefficient matricies for 4 frequencies

In [108]:
fig = plt.figure(figsize= (8,4))
tr = 141
for k in range(4):
    ax = plt.subplot(tr)
    tr+=1
    ax.imshow(np.real(A[k]))
    if (k == 0):
        ax.set_ylabel("Beam number")
        ax.set_xlabel("antennas")
    ax.set_title("Freq = {0:.4g}".format(freq[form_beams_with_freq[k]]))


plt.show()

In [15]:
'''
Plot Intensity as a function of source direction and beam number (2d plot)
'''
# %matplotlib tk
fig = plt.figure(figsize= (10,4))
plt.imshow(out.T)
plt.show()

In [12]:
plt.scatter(T_angles, P_angles, label = "Source Locations")
plt.scatter(theta, phi, label = "Beam Locations")
plt.legend()
plt.show()

## GPU Validation

This block compares the GPU implementation with the python implementation for a demonstration of correctness.
The GPU code exports a file called data.py which is stored in bin/. This code reads in that file and compares
it to the calculations made previously.

Note that to be accurate, N_AVERAGING, N_BEAMS, N_FREQS, etc. have to be the same for both implementations.

In [50]:
# import os, sys
try:
    import data
except:
    reload(data)
da = np.array(data.A)

vmx = max(np.max(data.A), np.max(out)) #determines scale

#Plot GPU image
ax = plt.subplot(131)
ax.set_title("GPU")
ax.set_ylabel("Source Direction")
ax.set_xlabel("Beam Number")

#plot Python image
ax2 = plt.subplot(132)
ax2.set_title("Python")
ax2.set_xlabel("Beam Number")
# ax2.colorbar()

#plot percent difference image
ax3 = plt.subplot(133)
ax3.set_title("Difference")
b = np.abs((out.T - da)/out.T)
plt.suptitle("Beamformer Validation")
ax3.set_xlabel("Beam Number")

im = ax.imshow(da, vmin = 0, vmax = vmx)
im2 = ax2.imshow(out.T, vmin = 0, vmax = vmx)
im3 = ax3.imshow(b*100, vmin = 0, vmax = 1)

fig.colorbar(im2, ax = ax2)
fig.colorbar(im3, ax = ax3)

plt.show()
print(np.sqrt(np.sum(b**2)/(1024*256)))
print(np.mean(b)*100)

0.0008414656773568963
0.038731603963115105


## Histograms

Plot histograms of both images and errors

In [32]:
ax = plt.subplot(121)
ax2 = plt.subplot(122)
_, bi, _ = ax.hist(x = np.log10(out.ravel()), bins=20, alpha = .7, log = True, label =  "Python")
ax.hist(x = np.log10(da.ravel()), bins=bi, alpha = .7, log = True, label = "GPU")
ax.set_title("Python vs GPU Image Histograms")
ax.set_xlabel("$\log_{10}$(Pixel Intensity)")
ax.legend()

ax2.hist(x = (b*100).ravel(), bins=20, alpha = .7, log = True)
ax2.set_title("Errors")
ax2.set_xlabel("% error")

plt.suptitle("Comparison of Beamformer Results")
plt.show()